In [ ]:
import random
import chainer 
import chainer.links as L 
import chainer.functions as F
from chainer.datasets import cifar
from chainer import optimizers
from chainer import iterators
from chainer import training
from chainer.training import extensions
from ipywidgets import interact
import numpy as np 
from matplotlib import pyplot as plt 
%matplotlib inline
from IPython.display import Image

In [ ]:
random.seed(0)
np.random.seed(0)
if chainer.cuda.available:
    chainer.cuda.cupy.random.seed(0)

In [ ]:
class MyNet(chainer.Chain):

    def __init__(self, n_out):
        super(MyNet, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 32, 3, 3, 1)
            self.conv2 = L.Convolution2D(32, 64, 3, 3, 1)
            self.conv3 = L.Convolution2D(64, 128, 3, 3, 1)
            self.fc4 = L.Linear(None, 1000)
            self.fc5 = L.Linear(1000, n_out)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        return h

In [ ]:
def train(network, 
          batchsize=128,
          device=0,
          max_epoch=20,
          train_dataset=None,
          test_dataset=None,
          postfix='',
          base_lr=0.01, 
          lr_decay=None):
    if train_dataset is None and test_dataset is None:
        train, test = cifar.get_cifar10()
    else:
        train, test = train_dataset, test_dataset 
    
    train_iter = iterators.MultiprocessIterator(train, batchsize)
    test_iter = iterators.MultiprocessIterator(test, batchsize, False, False)
    
    classifier = L.Classifier(network)
    
    optimizer = optimizers.MomentumSGD(lr=base_lr)
    optimizer.setup(classifier)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))
    
    updater = training.StandardUpdater(train_iter, optimizer, device=device)
    trainer = training.Trainer(updater,
                              (max_epoch, 'epoch'),
                               out='{}_cifar10_{}result'.format(network.__class__.__name__,postfix))
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.observe_lr())
    trainer.extend(extensions.Evaluator(test_iter, classifier, device=device), name='val')
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'val/main/loss', 'val/main/accuracy', 'elapsed_time', 'lr']))
    trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
    trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
    if lr_decay is not None:
        trainer.extend(extensions.ExponentialShift('lr',0.1), trigger=lr_decay)
    trainer.run()
    return classifier
    

In [ ]:
classifier = train(MyNet(10), device=0)

In [ ]:
Image(filename='MyNet_cifar10_result/loss.png')

In [ ]:
Image(filename='MyNet_cifar10_result/accuracy.png')

学習データでの精度（main/accuracy)は87%程度まで到達していますが、テストデータでのロス（val/main/loss）はむしろIterationを進むごとに大きくなってしまっており、またテストデータでの精度（val/main/accuracy）も60%前後で頭打ちになってしまっています。学習データでは良い精度が出ているが、テストデータでは精度が良くないということなので、モデルが学習データにオーバーフィッティングしていると思われます。

In [ ]:
klass_name = ['airplane', 'automobile', 'bird', 'cat', 'deer',
             'dog', 'frog', 'horse', 'ship', 'truck']
_, test = cifar.get_cifar10()

def predict(net, image_id):
    x, t = test[image_id]
    net.to_cpu()
    y = net.predictor(x[None, ...]).data.argmax(axis=1)[0]
    print('predicted_label:', klass_name[y])
    print('answer:', klass_name[t])
    plt.imshow(x.transpose(1,2,0))

In [ ]:
interact(lambda image_id:predict(classifier, image_id), image_id=list(range(len(test))))

In [ ]:
class ConvBlock(chainer.Chain):
    def __init__(self, n_ch, pool_drop=False):
        w = chainer.initializers.HeNormal()
        super(ConvBlock, self).__init__()
        with self.init_scope():
            self.conv=L.Convolution2D(None, n_ch, 3, 1, 1, nobias=True, initialW=w)
            self.bn = L.BatchNormalization(n_ch)
        self.pool_drop = pool_drop
        
    def __call__(self, x):
        h = F.relu(self.bn(self.conv(x)))
        if self.pool_drop:
            h=F.max_pooling_2d(h, 2, 2)
            h=F.dropout(h, ratio=0.25)
        return h 
    
class LinearBlock(chainer.Chain):
    def __init__(self, drop=True):
        w=chainer.initializers.HeNormal()
        super(LinearBlock, self).__init__()
        with self.init_scope():
            self.fc=L.Linear(None, 1024,initialW=w)
        self.drop = drop 
    def __call__(self, x):
        h = F.relu(self.fc(x))
        if self.drop:
            h = F.dropout(h)
        return h
    

In [ ]:
class DeepCNN(chainer.ChainList):

    def __init__(self, n_output):
        super(DeepCNN, self).__init__(
            ConvBlock(64),
            ConvBlock(64, True),
            ConvBlock(128),
            ConvBlock(128, True),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256, True),
            LinearBlock(),
            LinearBlock(),
            L.Linear(None, n_output)
        )

    def __call__(self, x):
        for f in self:
            x = f(x)
        return x

In [ ]:
model = train(DeepCNN(10), max_epoch=100, base_lr=0.1, lr_decay=(30, 'epoch'))

In [ ]:
Image(filename='DeepCNN_cifar10_result/loss.png')

In [ ]:
Image(filename='DeepCNN_cifar10_result/accuracy.png')

In [ ]:
class CIFAR10Augmented(chainer.dataset.DatasetMixin):

    def __init__(self, train=True):
        train_data, test_data = cifar.get_cifar10()
        if train:
            self.data = train_data
        else:
            self.data = test_data
        self.train = train
        self.random_crop = 4

    def __len__(self):
        return len(self.data)

    def get_example(self, i):
        x, t = self.data[i]
        if self.train:
            x = x.transpose(1, 2, 0)
            h, w, _ = x.shape
            x_offset = np.random.randint(self.random_crop)
            y_offset = np.random.randint(self.random_crop)
            x = x[y_offset:y_offset + h - self.random_crop,
                  x_offset:x_offset + w - self.random_crop]
            if np.random.rand() > 0.5:
                x = np.fliplr(x)
            x = x.transpose(2, 0, 1)

        return x, t

In [ ]:
model = train(DeepCNN(10), max_epoch=100, train_dataset=CIFAR10Augmented(), test_dataset=CIFAR10Augmented(False), postfix='augmented_', base_lr=0.1, lr_decay=(30, 'epoch'))

In [ ]:
Image(filename='DeepCNN_cifar10_augmented_result/loss.png')

In [ ]:
Image(filename='DeepCNN_cifar10_augmented_result/accuracy.png')

In [ ]:
from chainer.datasets import TransformDataset

train_dataset, test_dataset = cifar.get_cifar10()


# 行いたい変換を関数の形で書く
def transform(inputs):
    x, t = inputs
    x = x.transpose(1, 2, 0)
    h, w, _ = x.shape
    x_offset = np.random.randint(4)
    y_offset = np.random.randint(4)
    x = x[y_offset:y_offset + h - 4,
          x_offset:x_offset + w - 4]
    if np.random.rand() > 0.5:
        x = np.fliplr(x)
    x = x.transpose(2, 0, 1)

    return x, t


# 各データをtransformにくぐらせたものを返すデータセットオブジェクト
train_dataset = TransformDataset(train_dataset, transform)